In [1]:


import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from skimage.color import gray2rgb,rgb2gray,label2rgb
import cv2 as cv
import pandas as pd
import os
from tqdm import tqdm
import random
import pickle
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.applications import EfficientNetB0, DenseNet201,VGG16,VGG19,ResNet50,ResNet152V2,MobileNetV2,InceptionResNetV2

from sklearn.model_selection import train_test_split


SEED = 42



In [ ]:


TRAIN_PATH ='../input/chest-xray-pneumoniacovid19tuberculosis/train'
VAL_PATH = "../input/chest-xray-pneumoniacovid19tuberculosis/val"
TEST_PATH = "../input/chest-xray-pneumoniacovid19tuberculosis/test"



In [ ]:
print(f"Covid-19 X-Rays From Validation Set: {len(os.listdir(VAL_PATH + '/COVID19'))} ")
print(f"Normal X-Rays From Validation Set: {len(os.listdir(VAL_PATH + '/NORMAL'))} ")
print(f"Pneumonia X-Rays From Validation Set: {len(os.listdir(VAL_PATH + '/PNEUMONIA'))} ")
print(f"Tuberculosis X-Rays From Validation Set: {len(os.listdir(VAL_PATH + '/TURBERCULOSIS'))} ")



In [ ]:
train_covid19 = pd.DataFrame({"path": os.listdir(TRAIN_PATH + "/COVID19"), "label": "COVID19"})
train_covid19["path"] = train_covid19["path"].apply(lambda x: TRAIN_PATH + "/COVID19/" + x)



train_normal = pd.DataFrame({"path": os.listdir(TRAIN_PATH + "/NORMAL"), "label": "NORMAL"})
train_normal["path"] = train_normal["path"].apply(lambda x: TRAIN_PATH + "/NORMAL/" + x)

train_pneumonia = pd.DataFrame({"path": os.listdir(TRAIN_PATH + "/PNEUMONIA"), "label": "PNEUMONIA"})
train_pneumonia["path"] = train_pneumonia["path"].apply(lambda x: TRAIN_PATH + "/PNEUMONIA/" + x)
train_tuberculosis = pd.DataFrame({"path": os.listdir(TRAIN_PATH + "/TURBERCULOSIS"), "label": "TURBERCULOSIS"})
train_tuberculosis["path"] = train_tuberculosis["path"].apply(lambda x: TRAIN_PATH + "/TURBERCULOSIS/" + x)

train_df = pd.concat([train_covid19 ,train_normal, train_pneumonia,train_tuberculosis ])

val_covid19 = pd.DataFrame({"path": os.listdir(VAL_PATH + "/COVID19"), "label": "COVID19"})
val_covid19["path"] = val_covid19["path"].apply(lambda x: VAL_PATH + "/COVID19/" + x)
val_normal = pd.DataFrame({"path": os.listdir(VAL_PATH + "/NORMAL"), "label": "NORMAL"})
val_normal["path"] = val_normal["path"].apply(lambda x: VAL_PATH + "/NORMAL/" + x)
val_pneumonia = pd.DataFrame({"path": os.listdir(VAL_PATH + "/PNEUMONIA"), "label": "PNEUMONIA"})
val_pneumonia["path"] = val_pneumonia["path"].apply(lambda x: VAL_PATH + "/PNEUMONIA/" + x)
val_tuberculosis = pd.DataFrame({"path": os.listdir(VAL_PATH + "/TURBERCULOSIS"), "label": "TURBERCULOSIS"})
val_tuberculosis["path"] = val_tuberculosis["path"].apply(lambda x: VAL_PATH + "/TURBERCULOSIS/" + x)

val_df = pd.concat([val_covid19, val_normal, val_pneumonia,val_tuberculosis])

In [ ]:


test_covid19 = pd.DataFrame({"path": os.listdir(TEST_PATH + "/COVID19"), "label": "COVID19"})
test_covid19["path"] = test_covid19["path"].apply(lambda x: TEST_PATH + "/COVID19/" + x)

test_normal = pd.DataFrame({"path": os.listdir(TEST_PATH + "/NORMAL"), "label": "NORMAL"})
test_normal["path"] = test_normal["path"].apply(lambda x: TEST_PATH + "/NORMAL/" + x)


test_pneumonia = pd.DataFrame({"path": os.listdir(TEST_PATH + "/PNEUMONIA"), "label": "PNEUMONIA"})
test_pneumonia["path"] = test_pneumonia["path"].apply(lambda x: TEST_PATH + "/PNEUMONIA/" + x)


test_tuberculosis = pd.DataFrame({"path": os.listdir(TEST_PATH + "/TURBERCULOSIS"), "label": "TURBERCULOSIS"})
test_tuberculosis["path"] = test_tuberculosis["path"].apply(lambda x: TEST_PATH + "/TURBERCULOSIS/" + x)

test_df = pd.concat([test_covid19, test_normal, test_pneumonia, test_tuberculosis])



In [ ]:


train_data, val_data = train_test_split(train_df, 
                                        test_size = 0.1, 
                                        random_state = SEED, 
                                        stratify = train_df["label"], 
                                        shuffle = True)

val_data = pd.concat([val_df, val_data])

print(f"Training set size after re-splitting training data: {len(train_data)}")
print(f"Validation set size after re-splitting training data: {len(val_data)}")



In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize = (6, 6), facecolor = "#e5e5e5")
ax.set_facecolor("#e5e5e5")

sns.countplot(data = train_data, x = "label", ax = ax, color = "#101820")

ax.set_title("Countplot for Train Labels")

sns.despine()
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121,DenseNet169,DenseNet201, preprocess_input
train_datagen = ImageDataGenerator(
    brightness_range = (0.2, 1), 
    zoom_range = 0.2,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    preprocessing_function = preprocess_input
)

val_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input
)



In [ ]:
BATCH_SIZE = 16
train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_data,
    x_col = "path",
    y_col = "label",
    target_size = (224, 224),
    class_mode = "categorical",
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = SEED
)

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    x_col = "path",
    y_col = "label",
    target_size = (224, 224),
    class_mode = "categorical",
    batch_size = 1,
    shuffle = False,
    seed = SEED
)

In [ ]:
val_generator = val_datagen.flow_from_dataframe(
    dataframe = val_data,
    x_col = "path",
    y_col = "label",
    target_size = (224, 224),
    class_mode = "categorical",
    batch_size = BATCH_SIZE,
    shuffle = False,
    seed = SEED
)

In [ ]:
from vit_keras import vit, utils

image_size = 224
base_model = vit.vit_l16(
    image_size=image_size,
    activation='sigmoid',
    pretrained=True,
    include_top=True,
    pretrained_top=True,
     classes =8
    
)



In [ ]:


from tensorflow.keras.backend import categorical_crossentropy
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
def createmodel():
    model = Sequential()
    model.add(base_model) 
    model.add(BatchNormalization()),
    model.add(Dense(512,  kernel_initializer='lecun_normal',activation='gelu'))
    model.add(BatchNormalization()),
    model.add(Dense(4, activation='softmax', name ='output'))
  

    opt = tf.keras.optimizers.Adam(learning_rate=0.00001,beta_1=0.9,beta_2=0.999,epsilon=1e-07,amsgrad=False,name="Adam")

  
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2)

    model.compile(loss=loss, optimizer=opt, metrics=["accuracy"])
    return model



In [ ]:


model = createmodel()



In [ ]:


model.summary()



In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(
    monitor = "val_accuracy", 
    patience = 2,
    verbose = 1, 
    factor = 0.3, 
    min_lr = 0.000000001,
    cooldown = 1
)

history = model.fit(
    train_generator,
    epochs = 60, 
    batch_size = BATCH_SIZE,
    validation_data = val_generator,
    validation_steps = val_data.shape[0] // BATCH_SIZE,
    steps_per_epoch = train_data.shape[0] // BATCH_SIZE,
    callbacks = [reduce_lr]

    
)

In [ ]:


model.save('vit_l16.h5')



In [ ]:


num_label = {'COVID19': 0, 'NORMAL' : 1 , "PNEUMONIA":2, "TURBERCULOSIS":3}
Y_test = test_df['label'].copy().map(num_label).astype('int')



In [ ]:


import tensorflow as tf

model = tf.keras.models.load_model('/kaggle/input/modelhghgh/vit_l16.h5')

prediction=model.predict(test_generator,verbose=1)
print(prediction)



In [ ]:


yp=np.argmax(prediction,axis=1)



In [ ]:
from sklearn.metrics import classification_report,matthews_corrcoef

print(classification_report(Y_test, yp,digits = 4))
print(matthews_corrcoef(Y_test,yp))